In [ ]:
from twpasolver import MalnouBaseCell, StubBaseCell, TWPA
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

freqs = np.arange(1e9, 10e9, 1e6)

N_u= 60
N_l=6
N_sc=1000
N=N_sc*(N_u+N_l)

# Cell capacitances, expressed in F
C_u = 18.81e-15
C_l = 7.e-15
C_eff = (N_u * C_u + N_l * C_l)/(N_u+N_l)

# Cell inductances, expressed in H
L_u = 45.2e-12
L_l = 45.2e-12
L_eff = (N_u * L_u + N_l * L_l)/(N_u+N_l)

# Finger inductances, expressed in H
L_f_u = 1.02e-9
L_f_l = 0.335e-9
L_f_eff = (N_u * L_f_u + N_l * L_f_l)/(N_u+N_l)

# Characteristic inductance
Z_0 = np.sqrt(L_u / C_u)
Z_l = np.sqrt(L_l / C_l)
Z_eff = np.sqrt(L_eff / C_eff)

l1_u=102e-6
l1_l=33.5e-6
l2=2e-6

In [ ]:
u = MalnouBaseCell(C=C_u, L=L_u, Lf=L_f_u, Z0=Z_0)
l = MalnouBaseCell(C=C_l, L=L_l, Lf=L_f_l, Z0=Z_l)
eff = MalnouBaseCell(C=C_eff, L=L_eff, Lf=L_f_eff, Z0=Z_eff)
twpa=TWPA(unloaded=u, loaded=l, N_l=N_l, N_u=N_u, N_sc=N_sc, Z0=Z_0)
twpa_not_loaded=TWPA(unloaded=eff, loaded=eff, N_l=N_l, N_u=N_u, N_sc=N_sc, Z0=Z_eff)


# Stub model, TODO use better parameters
us = StubBaseCell(l1=l1_u, l2=l2, C=C_u, L=L_u, Lf=L_f_u, Z0=Z_0)
ls = StubBaseCell(l1=l1_l, l2=l2, C=C_l, L=L_l, Lf=L_f_l, Z0=Z_0)
twpa_stub=TWPA(unloaded=us, loaded=ls, N_l=N_l, N_u=N_u, N_sc=N_sc, Z0=Z_0)

### Simple dumping and loading of configuration files

In [ ]:
twpa.dump_to_file("model_1", writer="hdf5")
twpa2 = TWPA.from_file("model_1")
print(twpa2 == twpa)

### S matrix parameters

In [ ]:
cell = twpa.get_network(freqs)
cell_u = twpa_not_loaded.get_network(freqs)
cell_stub = twpa_stub.get_network(freqs)

In [ ]:
cell.s21.plot_s_db(label="malnou")
cell_stub.s21.plot_s_db(label="stub")
plt.grid()
plt.ylabel("$|S_{21}|$ (dB)")

### Non-linear dispersion relation

In [ ]:
cell = twpa.get_network(freqs)
linear_disp = 2*np.pi*freqs*np.sqrt(C_eff*L_eff)
plt.plot(freqs*1e-9, -cell.s21.s_rad_unwrap.flatten()/N - linear_disp, label="with loaded cells")
plt.plot(freqs*1e-9, -cell_u.s21.s_rad_unwrap.flatten()/N - linear_disp, label="no loaded cells")
plt.xlabel("Frequencies [GHz]")
plt.ylabel("$k^*$ [rad]")
plt.legend()
plt.grid()

### Phase-matching

In [ ]:
@njit
def phase_matching(freqs,k_array, pump_freqs,k_array_p, I0=250e-6, Istar=6.3e-3, I=1e-3):
    delta_k =  np.empty(shape=(len(freqs), len(pump_freqs)))
    for (i, j), _ in np.ndenumerate(delta_k):
        k_signal = k_array[i]
        k_pump = k_array_p[j]
        k_idler = k_array[np.searchsorted(freqs, pump_freqs[j]-freqs[i])]
        delta_k[i, j] = max(k_pump-k_signal-k_idler + I0**2 / (8*(I**2+Istar**2))*(k_pump-2*k_idler-2*k_signal),5e-4)
    return delta_k

all_freqs = np.arange(0, 10e9, 1e6)
cell = twpa.get_network(all_freqs)
all_k = cell.s21.s_rad_unwrap.flatten()/N

n_p_min =8500
n_p_max =-1

freqs_pump = all_freqs[n_p_min:n_p_max]
k_array_p = all_k[n_p_min:n_p_max]

n_s_max = 8500
freqs_s = all_freqs[:n_s_max]
k_array=all_k[:n_s_max]
linear_disp = 2*np.pi*freqs*np.sqrt(C_eff*L_eff)

deltas = phase_matching(freqs_s, k_array, freqs_pump, k_array_p)
linear_disp_pu = 2*np.pi*freqs_pump*np.sqrt(C_eff*L_eff)


In [ ]:
thin = 10
plt.pcolor(freqs_pump[::thin]*1e-9, freqs_s[::thin]*1e-9, deltas[::thin,::thin], norm="log")
c = plt.colorbar()
c.ax.set_xlabel(r"$\Delta_\beta$", fontsize=16)
plt.xlabel("pump frequency [GHz]")
plt.ylabel("signal frequency [GHz]")
#plt.savefig("phase_matching.png", dpi=150)